In [5]:
import joblib
import bz2
from urllib.parse import urlparse
from wikipedia2vec import Dictionary
import numpy as np

## Get all entity pairs

In [3]:
lang_dic = Dictionary.load('/mnt/local1/jinjin/de/dic')
eng_dic = Dictionary.load('/mnt/local1/jinjin/en/dic')

entity_pairs = []

with bz2.open('/data/jinjin/dbpedia/interlanguage_links_de.ttl.bz2', 'rt') as f:
    for i, line in enumerate(f):
        line = line.strip()
        
        if line.startswith('#'):
            continue
            
        lang, _, eng, _ = line.split()
        
        lang, eng = urlparse(lang[1:-1]), urlparse(eng[1:-1])
        
        if eng.netloc == 'dbpedia.org':
            lang_title = lang.path.split('/')[-1]
            eng_title = eng.path.split('/')[-1]
            
            lang_ent = lang_dic.get_entity(lang_title)
            eng_ent = eng_dic.get_entity(eng_title)
            
            if not lang_ent or not eng_ent:
                continue
            
            entity_pairs.append((eng_ent, lang_ent))
        
        if i % 1000000 == 0:
            print(i)

1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
11000000
12000000
13000000
14000000
15000000
16000000
17000000


## Take 1000 test pairs among frequent entities

In [6]:
freq_entity_pairs = list(filter(lambda x: x[0].count > 20 and x[1].count > 20, entity_pairs))
infreq_entity_pairs = list(filter(lambda x: x[0].count <= 20 or x[1].count <= 20, entity_pairs))
freq_entity_pairs = np.random.permutation(freq_entity_pairs)
test = list(freq_entity_pairs[:1000])
train = list(freq_entity_pairs[1000:]) + infreq_entity_pairs

In [7]:
joblib.dump(train, '/mnt/local1/jinjin/de/interlink')
with open('data/trans/en-de/entity/wiki.txt', 'w') as f:
    for eng_ent, lang_ent in test:
        f.write('{}\t{}\n'.format(eng_ent.title, lang_ent.title))